In [ ]:
import pandas as pd

import findlay2025a as f25a
import offproj as op  # See note below
import wisc_ecephys_tools as wet

In [2]:
nbsh = wet.get_sglx_project("seahorse")
experiment = wet.rats.constants.SleepDeprivationExperiments.NOD
sortings = f25a.units.get_nod_sortings()

In [3]:
strictness = "sua_moderate"
df = pd.read_parquet(nbsh.get_project_file(f"frs.with_meta.{strictness}.pqt"))

In [4]:
subjects = [subj for subj, _ in sortings]
for subject in subjects:
    hgs = f25a.hypnograms.load_statistical_condition_hypnograms(experiment, subject)
    for condition, hg in hgs.items():
        mask = (df["subject"] == subject) & (df["condition"] == condition)
        df.loc[mask, "condition_duration"] = hg["duration"].sum()
df["num_spikes"] = df["firing_rate"] * df["condition_duration"]

In [5]:
which = "ap"
mean_ap = False
by_channel = False
by_layer = False
spsl = op.get_completed_subject_probe_structure_list(
    ["Cx"], which=which, mean_ap=mean_ap, by_channel=by_channel, by_layer=by_layer
)
spsl = [
    (subj, prb, strct)
    for subj, prb, strct in spsl
    if (subj in subjects) and (subj in op.core.NOD_ANIMALS)
]

/home/gfindlay/projects/ece/wisc_ecephys_tools/src/wisc_ecephys_tools/rats/sortings.py:116: UserWarning: The following structures were unrecognized and ignored across all datasets: {'DG-dl-ml', 'CA3-DG-vl', 'CA1-so', 'CA1-sr', 'DG-dl-pol', '???', 'CA1-slm', 'CA3-DG', 'DG-vl-gcl', 'CA3-DG-dl', 'DG-pol', 'CA1-sp', 'DG-vl-ml', 'DG-dl-gcl', 'CA3-sp'}
  warnings.warn(


In [6]:
offs = []
for subject, probe, acronym in spsl:
    sps_offs = op.core.get_offs(
        subject,
        probe,
        acronym,
        layer=None,
        which=which,
        off_filters={
            "span": (100, float("Inf")),
            "median_duration": (0.03, 1.0),
            "duration": (0.0, 1.0),
        },
        experiment=experiment,
        mean_ap=mean_ap,
    )
    hgs = f25a.hypnograms.load_statistical_condition_hypnograms(experiment, subject)
    for condition, hg in hgs.items():
        is_condition = hg.covers_time(sps_offs["start_time"])
        _offs = sps_offs[is_condition].copy()
        _offs["condition"] = condition
        _offs["experiment"] = experiment
        _offs["subject"] = subject
        _offs["probe"] = probe
        _offs["acronym"] = acronym
        offs.append(_offs)

offs = pd.concat(offs, axis=0, ignore_index=True)

In [ ]:
# If you do not have offproj installed, you can find these off_durations included in
# the R package at findlay2025a/r/f25a/inst/extdata/off_durations.pqt
off_durations = offs[
    ["subject", "probe", "acronym", "condition", "duration"]
].to_parquet("off_durations.pqt")

In [8]:
spsc_off_durations = offs.groupby(["subject", "probe", "acronym", "condition"])[
    "duration"
].sum()

In [9]:
df = pd.merge(
    df,
    spsc_off_durations.rename("total_off_duration"),
    on=["subject", "probe", "acronym", "condition"],
    how="left",
)
df["on_firing_rate"] = df["num_spikes"] / (
    df["condition_duration"] - df["total_off_duration"]
)

In [10]:
df.to_parquet(nbsh.get_project_file(f"estm_on_frs.with_meta.{strictness}.pqt"))